In [1]:
pip install censusdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 49.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/7b/c6/7f75892d87d7afcf8ed909f3e74de1bc61abd9d77cd9aab1f449430856c5/charset_normalizer-3.2.0-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/9b/81/62fd61001fa4b9d0df6e31d47ff49cfa9de4af03adecf339c7bc30656b37/urllib3-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https:/

In [2]:
import pandas as pd
import censusdata

sample = censusdata.search('acs5', 2015, 'concept', 'transportation')

In [5]:
#trying this tutorial: https://www.natekratzer.com/posts/census_map/
import numpy as np
import geopandas as gpd
import requests

In [7]:
#reading in file and mapping counties using WI as example
county_shp = gpd.read_file('cb_2022_us_county_500k.zip')
wi_counties = county_shp[county_shp.STATE_NAME == 'Wisconsin']

In [9]:
wi_counties.explore()

In [18]:
#load API key
with open('census_api_key.txt') as key:
    api_key=key.read().strip()

#specify the data source by year and survey
year = '2020'
dsource = 'dec' #Decennial Census
dname = 'dhc' #Demographic and Housing Characteristics
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

#unique to this specific data request
cols = 'NAME,P1_001N' #NAME of geography as well as the variables I want to pull
geo = 'county:*' #county geography level
state = 'state:*' #all states

#add unique request features to the base_url
data_url = f'{base_url}?get={cols}&for={geo}&in={state}&key={api_key}'

#go get the data
response = requests.get(data_url)



In [19]:
#take the data in json and format it into a dataframe
data = response.json()

In [30]:
df = pd.DataFrame(data = data[1:], columns = data[0])
df.head()

,NAME,P1_001N,state,county
0,"Madera County, California",156255,06,039
1,"Marin County, California",262321,06,041
2,"Mariposa County, California",17131,06,043
3,"Mendocino County, California",91601,06,045
4,"Merced County, California",281202,06,047


In [39]:
#rename columns and convert population to numeric
df.rename(columns = {'NAME': 'county_name', 'P1_001N': 'population', 'state': 'state_fips', 'county': 'county_fips'}, inplace=True)
df['population'] = df['population'].apply(pd.to_numeric)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  3221 non-null   object
 1   population   3221 non-null   int64 
 2   state_code   3221 non-null   object
 3   county_code  3221 non-null   object
dtypes: int64(1), object(3)
memory usage: 100.8+ KB


In [ ]:
#matching population to geography data
